In [1]:
import os
os.chdir("..")

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from helpers import load_config


In [3]:
configs = load_config(os.path.join("configs", "configs.yaml"))

In [8]:
mutations_df = pd.read_csv(
    os.path.join(configs["STORAGE_DIR_STUDY2"], "data_mutations.txt"),
    sep="\t",
    header=0
)
mutations_df.head()

/var/folders/_0/qrjsyj4n0r951n63lt8qdzrc0000gn/T/ipykernel_72835/869686397.py:1: DtypeWarning: Columns (4,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  mutations_df = pd.read_csv(


,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,SYMBOL_SOURCE,TREMBL,TSL,UNIPARC,VARIANT_CLASS,all_effects,cDNA_position,n_depth,t_depth,Annotation_Status
0,DMBT1,1755.0,.,GRCh37,10,124399948,124399948,+,synonymous_variant,Silent,...,HGNC,B6V682_HUMAN,.,UPI000047021C,SNV,"DMBT1,synonymous_variant,p.%3D,ENST00000368956...",7054,115,131,SUCCESS
1,C10orf90,118611.0,.,GRCh37,10,128193330,128193330,+,missense_variant,Missense_Mutation,...,HGNC,"S4R3N7_HUMAN,Q5T025_HUMAN",.,UPI00001D808F,SNV,"C10orf90,missense_variant,p.Val100Met,ENST0000...",560,63,64,SUCCESS
2,WDFY4,57705.0,.,GRCh37,10,49997997,49997997,+,missense_variant,Missense_Mutation,...,HGNC,Q6PIM1_HUMAN,.,UPI000176ADB8,SNV,"WDFY4,missense_variant,p.Arg1345Trp,ENST000003...",4060,148,149,SUCCESS
3,PRKCQ,5588.0,.,GRCh37,10,6533674,6533674,+,missense_variant,Missense_Mutation,...,HGNC,.,.,UPI000012DF74,SNV,"PRKCQ,missense_variant,p.Gly254Val,ENST0000039...",861,197,228,SUCCESS
4,DYNC2H1,79659.0,.,GRCh37,11,103153768,103153768,+,missense_variant,Missense_Mutation,...,HGNC,.,.,UPI0000481AC7,SNV,"DYNC2H1,missense_variant,p.Pro3622Leu,ENST0000...",10865,204,199,SUCCESS


In [5]:
print(mutations_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332610 entries, 0 to 332609
Columns: 114 entries, Hugo_Symbol to Annotation_Status
dtypes: float64(3), int64(9), object(102)
memory usage: 289.3+ MB
None


In [6]:
print(mutations_df.columns)

Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'Center', 'NCBI_Build', 'Chromosome',
       'Start_Position', 'End_Position', 'Strand', 'Consequence',
       'Variant_Classification',
       ...
       'SYMBOL_SOURCE', 'TREMBL', 'TSL', 'UNIPARC', 'VARIANT_CLASS',
       'all_effects', 'cDNA_position', 'n_depth', 't_depth',
       'Annotation_Status'],
      dtype='object', length=114)


### Transforming data into one line per sample_id and all mutated genes comma-separated in the same line

In [9]:
mutations_df1 = mutations_df[["Hugo_Symbol", "Tumor_Sample_Barcode"]]

# Group by sample_id and aggregate mutations into a comma-separated string
grouped = mutations_df1.groupby('Tumor_Sample_Barcode')


mutations_df2 = grouped.agg(
    Mutation_Count=('Hugo_Symbol', 'count'),           # Define the name for the mutation count column
    Mutations=('Hugo_Symbol', lambda x: ', '.join(x))  # Define the name for the aggregated mutation column
).reset_index()

mutations_df2.head()

,Tumor_Sample_Barcode,Mutation_Count,Mutations
0,TCGA-3L-AA1B-01,185,"DMBT1, C10orf90, WDFY4, PRKCQ, DYNC2H1, SPON1,..."
1,TCGA-4N-A93T-01,151,"MYO3A, ARMC4, WAC, SVIL, KIAA1462, KIAA1462, K..."
2,TCGA-4T-AA8H-01,187,"CCDC147, LRRC37A6P, PCDH15, NRBF2, PIK3AP1, SL..."
3,TCGA-5M-AAT4-01,268,"PRLHR, PRLHR, ATE1, PTCHD3, SVIL, C10ORF68, PA..."
4,TCGA-5M-AAT5-01,134,"GPR123, ARHGAP21, VSTM4, CDH23, ITIH5, OR51S1,..."


In [10]:
mutations_df2.to_csv(
    os.path.join(configs['STORAGE_DIR_STUDY2'], 'mutations_summary.csv'), index=False)

### Using Gene2Vec-pretrained + GIT-finetuned gene embeddings to get a feature representation of each sample